In [90]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

Expected Loss for an individual is very less. So in bank expected loss is calculated according to Portfolio. Here I have individual information. So, here I have Calculated the expected Loss for all individuals.

## Calculation of LGD

In [91]:
loan_data_preprocessed=pd.read_csv('loan_preprocessed.csv')

c:\users\rhitc\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (21,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [94]:
loan_data_default=loan_data_preprocessed.copy()

In [95]:
loan_data_default.shape

(466285, 208)

In [96]:
loan_data_default['mths_since_last_delinq'].fillna(0,inplace=True)
loan_data_default['mths_since_last_record'].fillna(0,inplace=True)

In [97]:
features_all=['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'home_ownership:MORTGAGE',
'home_ownership:ANY',
'home_ownership:OTHER',
'home_ownership:NONE',
'home_ownership:RENT',
'home_ownership:OWN',
'purpose:debt_consolidation',
'purpose:credit_card',
'purpose:car', 
'purpose:home_improvement', 
'purpose:moving',
'purpose:educational', 
'purpose:medical',
'purpose:major_purchase',
'purpose:renewable_energy',
'purpose:other',
'purpose:wedding',
'purpose:vacation',
'purpose:small_business',
'initial_list_status:f',
'initial_list_status:w',
'emp_length_int',
'term_int',
'months_since_credit_Cr_line',
'months_issue_date',
'funded_amnt',
'int_rate',
'installment',
'annual_inc',
'dti',
'delinq_2yrs',
'inq_last_6mths',
'mths_since_last_record',
'mths_since_last_delinq',
'open_acc',
'pub_rec',
'total_acc',
'total_rev_hi_lim',
'acc_now_delinq'
]

In [98]:
features_ref_cat=['grade:G',
'verification_status:Verified',
'home_ownership:RENT',
'purpose:credit_card',
'initial_list_status:f'
]

In [99]:
loan_data_lgd_ead=loan_data_default[features_all]

In [100]:
loan_data_lgd_ead=loan_data_lgd_ead.drop(features_ref_cat,axis=1)

In [101]:
logit_reg_1=pickle.load(open('Logit_stage_1.sav','rb'))
lin_reg_2=pickle.load(open('Linear_model_stage_2.sav','rb'))

In [102]:
loan_data_default['recovery_rate_1']=logit_reg_1.predict(loan_data_lgd_ead)

In [103]:
loan_data_default['recovery_rate_2']=lin_reg_2.predict(loan_data_lgd_ead)

In [104]:
loan_data_default['recovery_rate']=loan_data_default['recovery_rate_1']*loan_data_default['recovery_rate_2']

In [105]:
loan_data_default['recovery_rate']=np.where(loan_data_default['recovery_rate']<0,0,loan_data_default['recovery_rate'])
loan_data_default['recovery_rate']=np.where(loan_data_default['recovery_rate']>1,1,loan_data_default['recovery_rate'])

In [106]:
loan_data_default['recovery_rate'].describe()

count    466285.000000
mean          0.076184
std           0.058970
min           0.000000
25%           0.000000
50%           0.096719
75%           0.126054
max           0.642424
Name: recovery_rate, dtype: float64

In [107]:
loan_data_default['LGD']=1-loan_data_default['recovery_rate']

## Calculation of EAD

In [108]:
reg_ead=pickle.load(open('reg_ead.sav','rb'))

In [110]:
loan_data_default['CCF']=reg_ead.predict(loan_data_lgd_ead)

In [111]:
loan_data_default['CCF']=np.where(loan_data_default['CCF']<0,0,loan_data_default['CCF'])
loan_data_default['CCF']=np.where(loan_data_default['CCF']>1,1,loan_data_default['CCF'])

In [112]:
loan_data_default['CCF'].describe()

count    466285.000000
mean          0.737595
std           0.101725
min           0.293121
25%           0.671053
50%           0.730688
75%           0.803395
max           1.000000
Name: CCF, dtype: float64

In [113]:
loan_data_default['EAD']=loan_data_default['CCF']*loan_data_default['funded_amnt']

In [114]:
loan_data_default['EAD'].describe()

count    466285.000000
mean      10814.989832
std        6935.008185
min         190.449515
25%        5495.638347
50%        9209.115052
75%       14692.003668
max       35000.000000
Name: EAD, dtype: float64

# Calculation of PD

In [45]:
x_train=pd.read_csv('input_train_with_ref_cat.csv')
y_train=pd.read_csv('Y_train.csv')
x_test=pd.read_csv('input_test_with_ref_cat.csv')
y_test=pd.read_csv('Y_test.csv')

In [44]:
ref_cat=['grade:G',
'homeOwnership_Rent_Other_Any_None',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'initial_list_status:f',
'term:60',
'emp_length:0',
'months_since_issue_date:>85',
'int_rate:>20.281',
'annual_inc:<20000',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'delinq_2yrs:>4',
'inq_last_6mths:>6',
'open_acc:0',
'pub_rec:0',
'total_acc:10-27',
'acc_now_delinq:0',
'months_since_credit_Cr_line:<140',
'total_rev_hi_lim:<=4k']

In [115]:
x_train.shape[0]+x_test.shape[0]

466285

In [51]:
x_train=x_train.drop(ref_cat,axis=1)
x_test=x_test.drop(ref_cat,axis=1)

## Setting Index

In [160]:
final_set.head()

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_VA,addr_state:UT_KY_AZ_NJ,...,total_acc:27-54,total_acc:>55,acc_now_delinq:>=1,months_since_credit_Cr_line:140-300,months_since_credit_Cr_line:>300,total_rev_hi_lim:4k-35k,total_rev_hi_lim:35k-56k,total_rev_hi_lim:56k-80k,total_rev_hi_lim:>80k,Intercept
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
427211,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,1,0,NaN
206088,0,0,1,0,0,0,0,1,0,1,...,1,0,0,1,0,1,0,0,0,NaN
136020,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,NaN
412305,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,NaN
36159,0,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,1,0,0,0,NaN


In [156]:
x_train=x_train.set_index('Unnamed: 0')
x_test=x_test.set_index('Unnamed: 0')

In [157]:
final_set=pd.concat([x_train,x_test],axis=0)

In [158]:
final_set.shape

(466285, 100)

In [159]:
pd_reg=pickle.load(open('pd_reg.sav','rb'))

In [161]:
final_set=final_set.drop(['Unnamed: 0','Intercept'],axis=1)

In [162]:
pd_reg.predict_proba(final_set)[:][:,1]

array([0.97283438, 0.90147733, 0.96461121, ..., 0.97461167, 0.95971959,
       0.95233288])

In [163]:
final_set['PD']=pd_reg.predict_proba(final_set)[:][:,1]

In [164]:
final_set['PD'].describe()

count    466285.000000
mean          0.890685
std           0.070716
min           0.348930
25%           0.853483
50%           0.906370
75%           0.943705
max           0.993522
Name: PD, dtype: float64

In [165]:
final_set['PD'].head()

Unnamed: 0
427211    0.972834
206088    0.901477
136020    0.964611
412305    0.804588
36159     0.835456
Name: PD, dtype: float64

# Combining DataFrames

In [166]:
loan_data_default['LGD'].head()

0    0.913958
1    0.915553
2    0.919489
3    0.904916
4    0.911430
Name: LGD, dtype: float64

In [167]:
final_set['PD'].head()

Unnamed: 0
427211    0.972834
206088    0.901477
136020    0.964611
412305    0.804588
36159     0.835456
Name: PD, dtype: float64

In [168]:
loan_data_default['EAD'].head()

0    2952.502664
1    1944.867422
2    1580.001275
3    6606.403694
4    2124.360153
Name: EAD, dtype: float64

In [170]:
loan_data_combined=pd.concat([loan_data_default['LGD'],loan_data_default['EAD'],final_set['PD']],axis=1)

In [171]:
loan_data_combined.shape

(466285, 3)

In [172]:
loan_data_combined.head()

,LGD,EAD,PD
0,0.913958,2952.502664,0.843747
1,0.915553,1944.867422,0.688957
2,0.919489,1580.001275,0.793156
3,0.904916,6606.403694,0.795975
4,0.911430,2124.360153,0.860539


In [173]:
loan_data_combined['EL']=loan_data_combined['PD']*loan_data_combined['LGD']*loan_data_combined['EAD']

In [174]:
loan_data_combined.head()

,LGD,EAD,PD,EL
0,0.913958,2952.502664,0.843747,2276.821743
1,0.915553,1944.867422,0.688957,1226.777005
2,0.919489,1580.001275,0.793156,1152.292874
3,0.904916,6606.403694,0.795975,4758.531463
4,0.911430,2124.360153,0.860539,1666.180511


 # Epected loss from the Individual Protfolio is 

In [175]:
loan_data_combined['EL'].sum()

4136088861.5519786